In [25]:
!pip install UQpy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/917.8 kB ? eta -:--:--
   --------------------------------------- 917.8/917.8 kB 10.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114262 sha256=f0b04ea5fb588023d2a22183bc5a7db7a6e3cf238cb562bc1a24dc1c61c67283
  Stored in directory: c:\users\trago\appdata\local\pip\cache\wheels\9e\5b\45\29f72e55d87a29426b04b3cfdf20325c079eb97ab74f59017d
Successfully built fire


In [4]:
import numpy as np
from UQpy.surrogates import *
from UQpy.distributions import Uniform, JointIndependent,Normal,Lognormal
from UQpy.sensitivity import PceSensitivity

class PolynomialChaosExpansionUQPY():
    def __init__(self, pdf, exp_design_in, exp_design_out, degree=4):
        self.pdf = pdf if pdf is not None else JointIndependent([Uniform(0, 1) for _ in range(exp_design_in.shape[1])])
        self.num_inputs = exp_design_in.shape[1]
        self.num_outputs = exp_design_out.shape[1]
        self.num_samples = exp_design_out.shape[0]
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out
        
        self.polynomial_basis = TotalDegreeBasis(self.pdf, degree)
        self.regression_method = LeastSquareRegression()
        self.pce = PolynomialChaosExpansion(polynomial_basis=self.polynomial_basis, regression_method=self.regression_method)
        self.pce.fit(exp_design_in, exp_design_out)
    
    def predict(self, design_in):
        return self.pce.predict(design_in)
        
    def compute_mean(self):
        mean_value, _ = self.pce.get_moments(higher=False)
        return mean_value
    
    def compute_variance(self):
        _, variance_value = self.pce.get_moments(higher=False)
        return variance_value
    
    def compute_sobol_first(self):
        sensitivity = PceSensitivity(self.pce)
        sensitivity.run()
        return sensitivity.first_order_indices
    
    def compute_sobol_total(self):
        sensitivity = PceSensitivity(self.pce)
        sensitivity.run()
        return sensitivity.total_order_indices

# Zacatek volani PCE
input_sample = np.array([
    [0.0, 0.0],
    [0.5, 0.2],
    [1.0, 0.4],
    [1.5, 0.6],
    [2.0, 0.8],
    [2.5, 1.0],
    [3.0, 1.2],
    [3.5, 1.4],
    [4.0, 1.6],
    [4.5, 1.8]
])

def test_function(X):
    x1, x2 = X[:, 0], X[:, 1]
    return np.expand_dims(2*x1 + 3*x2**2, axis=1)  # Skalární výstup

output_sample = test_function(input_sample)

# Definice vstupů 
marginals = [Uniform(0, 5), Uniform(0, 2)] 
input_distribution = JointIndependent(marginals)

pce_uqpy = PolynomialChaosExpansionUQPY(input_distribution, input_sample, output_sample)

# Predikce na testovacích datech
test_inputs = np.array([
    [0.25, 0.1],
    [0.75, 0.3],
    [1.25, 0.5],
    [1.75, 0.7],
    [2.25, 0.9]
])
predictions = pce_uqpy.predict(test_inputs)

mean_value = pce_uqpy.compute_mean()
variance_value = pce_uqpy.compute_variance()

sobol_first = pce_uqpy.compute_sobol_first()
sobol_total = pce_uqpy.compute_sobol_total()

# Výpis výsledků
print("První řád Sobolových indexů:")
print(sobol_first)
print("Celkové Sobolovy indexy:")
print(sobol_total)
print("Střední hodnota modelu:")
print(mean_value)
print("Rozptyl modelu:")
print(variance_value)

true_outputs = test_function(test_inputs)
print("Skutečné hodnoty vs Predikované hodnoty:")
for i in range(len(test_inputs)):
    print(f"Vstupy: {test_inputs[i]}, Skutečná hodnota: {true_outputs[i][0]}, Predikovaná hodnota: {predictions[i][0]}")



První řád Sobolových indexů:
[[0.19959207]
 [0.19959207]]
Celkové Sobolovy indexy:
[[0.80040793]
 [0.80040793]]
Střední hodnota modelu:
3.5004115226337493
Rozptyl modelu:
32.42325085945578
Skutečné hodnoty vs Predikované hodnoty:
Vstupy: [0.25 0.1 ], Skutečná hodnota: 0.53, Predikovaná hodnota: 0.5300000000000198
Vstupy: [0.75 0.3 ], Skutečná hodnota: 1.77, Predikovaná hodnota: 1.7700000000000409
Vstupy: [1.25 0.5 ], Skutečná hodnota: 3.25, Predikovaná hodnota: 3.2500000000000266
Vstupy: [1.75 0.7 ], Skutečná hodnota: 4.97, Predikovaná hodnota: 4.970000000000005
Vstupy: [2.25 0.9 ], Skutečná hodnota: 6.93, Predikovaná hodnota: 6.929999999999993
